In [1]:
from numpy import linalg as LA
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks_cwt
import pickle
from pathlib import Path
from scipy.stats import kurtosis, skew
from scipy.stats import iqr
from pyentrp import entropy as ent
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
import sys
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt

%matplotlib inline

In [35]:
in_dir = "test3"

## Compute features for accelerometer only

In [ ]:
trim_num_seconds = 10
acc_freq = 4
window_num_seconds = 4 #seconds
steps_per_sec = int(1000/acc_freq)
window_size = 10#int(window_num_seconds*steps_per_sec)
window_step = 2 #seconds
window_jump_steps = 5#int(window_step*steps_per_sec)

print("Window_size, Window_jump_steps: ", window_size, window_jump_steps)

#Helper Functions

def Signal_magnitude_area(x,y,z):
    
    sum = 0    
    for i in range(len(x)):
        sum += (abs(x[i]) + abs(y[i]) + abs(z[i]))
        
    return float(sum)/len(x)


def Power(x):
    
    power = (LA.norm(x)**2)/ len(x)
    return power
    
def number_of_peaks(window):
    indexes = find_peaks_cwt(window, np.arange(1, len(window)))

    return len(indexes)

#this function assumes that records are evenly spaced
def trim_first_last_n_seconds(df, n, freq):
    if df.shape[0] < 6001:
        return None
    
    remove_indexes = list(range(0, int(n*1000/freq)))
    df = df.drop(remove_indexes)

    remove_indexes = list(range(df.shape[0] - int(n*1000/freq), df.shape[0]-1))
    df = df.drop(remove_indexes)
    
    return df
    
def load_files(indir,pickles_indir):
    dfs_list = []
    features = []
    labels = []
    bypass = True
    pickle_file = Path(pickles_indir+"/img_accelerometer_features.pickle")

    if pickle_file.exists() and not bypass:
        print("Found pickle files for accelerometer")
        features = pickle.load(open(pickles_indir+"/img_accelerometer_features.pickle", "rb"))
        labels = pickle.load(open(pickles_indir+"/img_accelerometer_labels.pickle", "rb"))
        dfs_list = pickle.load(open(pickles_indir+"/img_accelerometer_dfs_list.pickle", "rb"))

    else:

        for root, dirs, files in os.walk(in_dir):
            path = root.split(os.sep)
            for f in files:
                print("/".join(path) + "/" + f)
                full_path = "/".join(path) + "/" + f
                if "gyroscope" in full_path:
                    print("Skip ", full_path)
                    continue
                df = pd.read_csv(full_path, header=None)

                print("Before trimming: ", df.shape)

                df = trim_first_last_n_seconds(df, trim_num_seconds, acc_freq)
                if df is None:
                    print("Continuing")
                    continue

                print("After trimming: ", df.shape)

                #Sample the data according to the size of the window with 50% overlap
                for index in range(0, df.shape[0]-window_size, window_jump_steps):
                    indexes = list(range(index, index + int(window_size)))

                    window = df.iloc[indexes,:]

                    X_list = window[1].tolist()
                    Y_list = window[2].tolist()
                    Z_list = window[3].tolist()


                    #Generate the features for this window


           # ****************** Time-Domain Features ************************* #

                    #Mean of the signals
                    mean_x = np.mean(X_list)
                    mean_y = np.mean(Y_list)
                    mean_z = np.mean(Z_list)

                    #Variance of the signals
                    var_x = np.var(X_list)
                    var_y = np.var(Y_list)
                    var_z = np.var(Z_list)

                    #Number of peaks in the signals
                    #num_peaks_x = number_of_peaks(X_list)
                    #num_peaks_y = number_of_peaks(Y_list)
                    #num_peaks_z = number_of_peaks(Z_list)            

                    #Median of the signals
                    median_x = np.ma.median(X_list)
                    median_y = np.ma.median(Y_list)
                    median_z = np.ma.median(Z_list)

                    #Standard Deviation of the signals
                    std_x = np.std(X_list)
                    std_y = np.std(Y_list)
                    std_z = np.std(Z_list)

                    #Compute Signal Magnitude Area
                    signal_mag_area = Signal_magnitude_area(X_list, Y_list, Z_list)

                    #Maximum and Minimum values and their indexes
                    max_x = max(X_list)
                    max_index_x = X_list.index(max_x)               
                    min_x = min(X_list)
                    min_index_x = X_list.index(min_x)

                    max_y = max(Y_list)
                    max_index_y = Y_list.index(max_y)              
                    min_y = min(Y_list)
                    min_index_y = Y_list.index(min_y)               

                    max_z = max(Z_list)
                    max_index_z = Z_list.index(max_z)             
                    min_z = min(Z_list)
                    min_index_z = Z_list.index(min_z)


                    #Power of X,Y and Z signals             
                    power_x = Power(X_list)
                    power_y = Power(Y_list)
                    power_z = Power(Z_list)


                    #Skewness and Kurtosis
                    skew_x = skew(X_list)
                    skew_y = skew(Y_list)
                    skew_z = skew(Z_list)

                    kurtosis_x = kurtosis(X_list)                
                    kurtosis_y = kurtosis(Y_list)
                    kurtosis_z = kurtosis(Z_list)


                    #Entropy of the signals (Can experiment with different types of Entropy)
                    entropy_x = ent.shannon_entropy(X_list)
                    entropy_y = ent.shannon_entropy(Y_list)
                    entropy_z = ent.shannon_entropy(Z_list)


                    #Interquartile range of the signals
                    iqr_x = iqr(X_list)
                    iqr_y = iqr(Y_list)
                    iqr_z = iqr(Z_list)


        # ****************** Frequency-Domain Features ************************* #

                    #Normalized FFT coefficients
                    fft_x = LA.norm(np.fft.rfft(X_list))              
                    fft_y = LA.norm(np.fft.rfft(Y_list))   
                    fft_z = LA.norm(np.fft.rfft(Z_list))  


                    #Store the features
                    window_feature = []
                    window_feature.append(mean_x)
                    window_feature.append(mean_y)
                    window_feature.append(mean_z)

                    window_feature.append(var_x)
                    window_feature.append(var_y)
                    window_feature.append(var_z)

                    window_feature.append(median_x)
                    window_feature.append(median_y)
                    window_feature.append(median_z)

                    window_feature.append(std_x)
                    window_feature.append(std_y)
                    window_feature.append(std_z)


                    window_feature.append(signal_mag_area)

                    window_feature.append(max_x)
                    window_feature.append(max_index_x)
                    window_feature.append(min_x)
                    window_feature.append(min_index_x)

                    window_feature.append(max_y)
                    window_feature.append(max_index_y)
                    window_feature.append(min_y)
                    window_feature.append(min_index_y)

                    window_feature.append(max_z)
                    window_feature.append(max_index_z)
                    window_feature.append(min_z)
                    window_feature.append(min_index_z)

                    window_feature.append(power_x)
                    window_feature.append(power_y)
                    window_feature.append(power_z)

                    window_feature.append(skew_x)
                    window_feature.append(kurtosis_x) 

                    window_feature.append(skew_y)
                    window_feature.append(kurtosis_y) 

                    window_feature.append(skew_z)
                    window_feature.append(kurtosis_z) 

                    window_feature.append(entropy_x)
                    window_feature.append(entropy_y)
                    window_feature.append(entropy_z)

                    window_feature.append(iqr_x)
                    window_feature.append(iqr_y)
                    window_feature.append(iqr_z)

                    window_feature.append(fft_x)
                    window_feature.append(fft_y)
                    window_feature.append(fft_z)


                    #window_feature.append(num_peaks_x)
                    #window_feature.append(num_peaks_y)
                    #window_feature.append(num_peaks_z)

                    #scale = preprocessing.minmax_scale(data, feature_range=(-0.5, 0.5))

                    features.append(window_feature)

                    labels.append(df[5].iloc[1])
                dfs_list.append(df)
        dfs = pd.concat(dfs_list)

        #pickle.dump(features, open("pickles/img_accelerometer_features.pickle", "wb"), protocol=2)
        #pickle.dump(labels, open("pickles/img_accelerometer_labels.pickle", "wb"), protocol=2)
        #pickle.dump(dfs_list, open("pickles/img_accelerometer_dfs_list.pickle", "wb"), protocol=2)

    features = np.asarray(features)
    labels = np.asarray(labels)
    return features, labels

### Loading test and train features from appropriate directories

In [4]:
features_train, labels_train = load_files(indir='data-3',pickles_indir='img_pickles_train')
features_test, labels_test = load_files(indir='test3',pickles_indir='img_pickles_test')

(1183508, 43)
(1183508,)


In [ ]:
## Using LDA to reduce the fimensions into 3 

In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=3)

In [6]:
features_train = lda.fit(features_train, labels).transform(features_train)
features_test = lda.transform(features_test)

In [7]:
print(features_train.shape)
print(labels_train.shape)
print(features_test.shape)
print(labels_test.shape)

(1183508, 3)
(1183508,)


## Put the train and test features into dataframe

In [8]:
features_df_train = pd.DataFrame(features_train,columns=['a','b','c'])
labels_df_train = pd.DataFrame(labels_train,columns=['l'])
features_df_test = pd.DataFrame(features_test,columns=['a','b','c'])
labels_df_test = pd.DataFrame(labels_test,columns=['l'])

In [9]:
features_df_train['l'] = labels_df_train['l']
features_df_test['l'] = labels_df_test['l']

In [10]:
activities = ['walking','sitting','standing','laying_down']

## Findind the minimum and maximum using 1 stddev away from mean

In [16]:
max_=[]
min_=[]
for axis in ['a','b','c']:
    print('axis',axis)
    mean =np.mean(features_df_train[axis])
    std =np.std(features_df_train[axis]) 
    print(mean,std)
    print(np.min(features_df_train[axis]),np.max(features_df_train[axis]))
    max_.append(mean+2*std)
    min_.append(mean-2*std)
    #max_.append()        


('axis', 'a')
(5.772392346708159e-12, 1.9192292699777684)
(-31.621895458060305, 10.54334094324393)
('axis', 'b')
(-7.767921600725771e-13, 1.2881504023883432)
(-32.04581690334954, 14.016145053606209)
('axis', 'c')
(1.608658157529988e-13, 1.146049195629901)
(-38.52436291103095, 8.237474300467811)


In [39]:
max_

[10.54334094324393, 14.016145053606209, 8.237474300467811]

In [40]:
min_

[-31.621895458060305, -32.04581690334954, -38.52436291103095]

## Functions to scale features from feature values to colrs

In [19]:
def get_color(i,num):
    scale_ = 255 /(max_[i]-min_[i])
    num = max(min_[i],num)
    num = min(num,max_[i])
    return (num - min_[i])*scale_
def get_rgb(feature):
    #print(feature)
    feature = list(feature)
    #print(feature)
    col = []
    for i in range(0,3):
        col.append(int(get_color(i,feature[i])))
    #col[1] = 0
    return col
    

In [20]:
def spiral(lst,n):
    dx,dy = 1,0            # Starting increments
    x,y = 0,0              # Starting location
    myarray = [[None]* n for j in range(n)]
    for i in range(n**2):
        myarray[x][y] = lst[i]
        nx,ny = x+dx, y+dy
        if 0<=nx<n and 0<=ny<n and myarray[nx][ny] == None:
            x,y = nx,ny
        else:
            dx,dy = -dy,dx
            x,y = x+dx, y+dy
    return myarray

## Features converted to 0 - 255

In [ ]:
array_map_train={}
for activity in activities:
    print('train')
    df = features_df_train[features_df_train['l']==activity][['a','b','c']].copy()
    print(df.shape)
    array_map_train[activity] = (df.as_matrix())
    print((array_map_train[activity][2]))
    print('test')
    df = features_df_test[features_df_test['l']==activity][['a','b','c']].copy()
    print(df.shape)
    array_map_test[activity] = (df.as_matrix())
    print((array_map_test[activity][2]))
    

In [22]:
color_map_train = {}
color_map_test = {}
for activity in activities:
    color_map_train[activity] = [get_rgb(x) for x in array_map_train[activity]]
    print(color_map_train[activity][1])
    color_map_test[activity] = [get_rgb(x) for x in array_map_test[activity]]
    print(color_map_test[activity][1])

[64, 255, 37]
[54, 122, 166]
[200, 80, 65]
[187, 79, 205]


## Coverting to square images

In [ ]:
from collections import defaultdict
color_map_spiral_train = defaultdict(list)
color_map_spiral_test = defaultdict(list)
for activity in activities:
    print(activity)
    siz = len(color_map_train[activity])
    for i in range((siz/625)-1):
        sp = spiral(color_map_train[activity][i*625:(i+1)*625],25)
        color_map_spiral_train[activity].append(sp)
    print(len(color_map_spiral_train[activity]))
    siz = len(color_map_test[activity])
    for i in range((siz/625)-1):
        sp = spiral(color_map_test[activity][i*625:(i+1)*625],25)
        color_map_spiral_test[activity].append(sp)
    print(len(color_map_spiral_test[activity]))    

## Storing the images

In [24]:
from PIL import Image
import numpy as np

In [25]:
train_dir = 'img_train'
test_dir = 'img_test'
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

for activity in activities:
    i=0
    print(activity)
    if not os.path.exists(train_dir+'/'+activity):
        os.mkdir(train_dir+'/'+activity)        
    if not os.path.exists(test_dir+'/'+activity):
        os.mkdir(test_dir+'/'+activity)
        
    for spir_arr in color_map_spiral_train[activity]:
        i+=1
        img = Image.fromarray(np.array(spir_arr), 'RGB')
        img.save(train_dir+'/'+activity+'/'+activity+str(i)+'.png')
    
    for spir_arr in color_map_spiral_test[activity]:
        i+=1
        img = Image.fromarray(np.array(spir_arr), 'RGB')
        img.save(test_dir+'/'+activity+'/'+activity+str(i)+'.png')


laying_down
sitting
standing
walking


# TEST

In [33]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# dimensions of our images.
img_width, img_height = 25, 25

train_data_dir = 'CNN/train'
validation_data_dir = 'CNN/test'
test_data_dir = 'CNN/actual_testing'

nb_train_samples = 1711
nb_validation_samples = 178
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

    
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 1711 images belonging to 4 classes.
Found 178 images belonging to 4 classes.
Epoch 1/50
106/106 [==============================] - 3s 28ms/step - loss: 1.2303 - acc: 0.4083 - val_loss: 0.9308 - val_acc: 0.5852
Epoch 2/50
106/106 [==============================] - 2s 17ms/step - loss: 0.9811 - acc: 0.5575 - val_loss: 0.7358 - val_acc: 0.6307
Epoch 3/50
106/106 [==============================] - 2s 17ms/step - loss: 0.8743 - acc: 0.6189 - val_loss: 0.7507 - val_acc: 0.6364
Epoch 4/50
106/106 [==============================] - 2s 17ms/step - loss: 0.8098 - acc: 0.6725 - val_loss: 0.5440 - val_acc: 0.7955
Epoch 5/50
106/106 [==============================] - 2s 18ms/step - loss: 0.7610 - acc: 0.7032 - val_loss: 0.4706 - val_acc: 0.8125
Epoch 6/50
106/106 [==============================] - 2s 17ms/step - loss: 0.7289 - acc: 0.7309 - val_loss: 0.4222 - val_acc: 0.8352
Epoch 7/50
106/106 [==============================] - 2s 18ms/step - loss: 0.6928 - acc: 0.7446 - val_loss: 0.4249 - va

In [107]:
label_map = (train_generator.class_indices)
label_map

{'laying_down': 0, 'sitting': 1, 'standing': 2, 'walking': 3}

In [108]:
test_data_dir = 'CNN/actual_testing'

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    #batch_size=batch_size,
    class_mode='categorical')

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator)

Found 260 images belonging to 4 classes.


In [ ]:
train_generator.filenames

In [76]:
y_test = [[1,0,0,0]]*len(color_map_spiral['laying_down'])+[[0,1,0,0]]*len(color_map_spiral['sitting'])+[[0,0,1,0]]*len(color_map_spiral['standing'])+[[0,0,0,1]]*len(color_map_spiral['walking'])

In [116]:
#print predict

def get_hot_value(my_list):
        max_val = max(my_list)
        return [int(item == max_val) for item in my_list]

hot_list = [get_hot_value(sublist) for sublist in predict]

t  = np.array(y_test)
p  = np.array(hot_list)
t = np.argmax(t, axis=1)
p = np.argmax(p, axis=1)

#print hot_list
print("Precision: ", precision_score(t, p, average=None))

from sklearn.metrics import accuracy_score
print accuracy_score(t, p)

('Precision: ', array([0.48051948, 0.37209302, 0.14705882, 0.125     ]))
0.27692307692307694


In [105]:
print len(predict)
print len(y_test)

260
260


In [110]:
predict

array([[1.0990019e-07, 4.2163174e-05, 1.4948982e-02, 8.5364991e-01],
       [3.1117118e-09, 4.0054915e-06, 2.2853166e-03, 9.4797361e-01],
       [2.6103375e-09, 4.0909013e-06, 3.2966968e-03, 9.4010174e-01],
       ...,
       [5.2023053e-01, 1.7847300e-02, 9.1647869e-04, 8.2183027e-07],
       [6.1969858e-01, 9.8320469e-03, 5.1267534e-02, 7.5963067e-06],
       [4.1519919e-01, 2.3386780e-02, 2.6436889e-01, 9.7757904e-05]],
      dtype=float32)

In [ ]:
hot_list